In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

## Pre-processing

In [2]:
DATAPATH = 'tadpole_challenge/TADPOLE_D1_D2.csv'
df = pd.read_csv(DATAPATH)

D:\Anaconda2\envs\py38\lib\site-packages\IPython\core\interactiveshell.py:3062: DtypeWarning: Columns (471,473,474,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,569,570,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,599,601,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,624,625,626,627,628,629,630,631,632,633,634,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,7

In [3]:
getting_started = ["RID", "VISCODE", "DX_bl",
                  "DX", "ADAS13", "Ventricles",
                  "CDRSB", "ADAS11", "MMSE", "RAVLT_immediate",
                  "Hippocampus", "WholeBrain", "Entorhinal", "MidTemp",
                  "FDG", "AV45",
                  "ABETA_UPENNBIOMK9_04_19_17", "TAU_UPENNBIOMK9_04_19_17", "PTAU_UPENNBIOMK9_04_19_17",
                  "APOE4", "AGE"]

In [16]:
# Use features from the 'Getting Started' section of the Tadpole website
data = df[getting_started]
# Only use baseline visits
data = data.loc[data['VISCODE'] == 'bl']
print('#baseline visits:', data.shape[0])
data.head()

#baseline visits: 1737


,RID,VISCODE,DX_bl,DX,ADAS13,Ventricles,CDRSB,ADAS11,MMSE,RAVLT_immediate,...,WholeBrain,Entorhinal,MidTemp,FDG,AV45,ABETA_UPENNBIOMK9_04_19_17,TAU_UPENNBIOMK9_04_19_17,PTAU_UPENNBIOMK9_04_19_17,APOE4,AGE
0,2,bl,CN,NL,18.67,118233.0,0.0,10.67,28.0,44.0,...,1229740.0,4177.0,27936.0,1.36926,NaN,,,,0.0,74.3
1,3,bl,AD,Dementia,31.00,84599.0,4.5,22.00,20.0,22.0,...,1129830.0,1791.0,18422.0,1.09079,NaN,741.5,239.7,22.83,1.0,81.3
5,4,bl,LMCI,MCI,21.33,39605.0,1.0,14.33,27.0,37.0,...,1154980.0,3983.0,19615.0,NaN,NaN,1501,153.1,13.29,0.0,67.5
10,5,bl,CN,NL,14.67,34062.0,0.0,8.67,29.0,37.0,...,1116630.0,4433.0,21614.0,1.29799,NaN,547.3,337,33.43,0.0,73.7
15,6,bl,LMCI,MCI,25.67,39826.0,0.5,18.67,25.0,30.0,...,927510.0,2277.0,17802.0,NaN,NaN,,,,0.0,80.4


In [17]:
data.dtypes

RID                             int64
VISCODE                        object
DX_bl                          object
DX                             object
ADAS13                        float64
Ventricles                    float64
CDRSB                         float64
ADAS11                        float64
MMSE                          float64
RAVLT_immediate               float64
Hippocampus                   float64
WholeBrain                    float64
Entorhinal                    float64
MidTemp                       float64
FDG                           float64
AV45                          float64
ABETA_UPENNBIOMK9_04_19_17     object
TAU_UPENNBIOMK9_04_19_17       object
PTAU_UPENNBIOMK9_04_19_17      object
APOE4                         float64
AGE                           float64
dtype: object

In [18]:
# Inspect columns with dtype 'object'
col_dtype_object = [col for col in data.columns if data[col].dtype == object]

col_wrong_dtype = []
for col in col_dtype_object:
    print(col, '\n', pd.unique(data[col]), '\n')
    if len(pd.unique(data[col])) > 100:
        col_wrong_dtype.append(col)
        
print('Possible wrong dtypes:\n', col_wrong_dtype)

VISCODE 
 ['bl'] 

DX_bl 
 ['CN' 'AD' 'LMCI' 'EMCI' 'SMC'] 

DX 
 ['NL' 'Dementia' 'MCI' 'Dementia to MCI' 'MCI to Dementia' 'MCI to NL' nan
 'NL to MCI'] 

ABETA_UPENNBIOMK9_04_19_17 
 [' ' '741.5' '1501' ... '558.9' '416.8' '1460'] 

TAU_UPENNBIOMK9_04_19_17 
 [' ' '239.7' '153.1' ... '268.4' '88.69' '224.9'] 

PTAU_UPENNBIOMK9_04_19_17 
 [' ' '22.83' '13.29' ... '26.11' '8.53' '20.9'] 

Possible wrong dtypes:
 ['ABETA_UPENNBIOMK9_04_19_17', 'TAU_UPENNBIOMK9_04_19_17', 'PTAU_UPENNBIOMK9_04_19_17']


In [19]:
# count = 0
# for col in col_wrong_dtype:
#     data[col] = data[col_wrong_dtype[0]].replace(r'^\s*$', np.nan, regex=True)
#     data[col] = data[col].astype(np.float64)
#     print(col, data[col].isna().sum())

# Drop these columns for now (weird values e.g., ' ', '<200')
data = data.drop(columns=col_wrong_dtype)

In [20]:
# Convert NaNs from float columns to 0
col_dtype_float = [col for col in data.columns if data[col].dtype == np.float64 or data[col].dtype == np.int64]
data[col_dtype_float] = data[col_dtype_float].fillna(0)
data = data.dropna() # drop remaining NaNs from categorical variables

# Convert categorical to numeric with sklearn's LabelEncoder
col_dtype_object = [col for col in data.columns if data[col].dtype == object]
data[col_dtype_object] = data[col_dtype_object].apply(LabelEncoder().fit_transform)
data

,RID,VISCODE,DX_bl,DX,ADAS13,Ventricles,CDRSB,ADAS11,MMSE,RAVLT_immediate,Hippocampus,WholeBrain,Entorhinal,MidTemp,FDG,AV45,APOE4,AGE
0,2,0,1,5,18.67,118233.0,0.0,10.67,28.0,44.0,8336.0,1229740.0,4177.0,27936.0,1.36926,0.00000,0.0,74.3
1,3,0,0,0,31.00,84599.0,4.5,22.00,20.0,22.0,5319.0,1129830.0,1791.0,18422.0,1.09079,0.00000,1.0,81.3
5,4,0,3,2,21.33,39605.0,1.0,14.33,27.0,37.0,6869.0,1154980.0,3983.0,19615.0,0.00000,0.00000,0.0,67.5
10,5,0,1,5,14.67,34062.0,0.0,8.67,29.0,37.0,7075.0,1116630.0,4433.0,21614.0,1.29799,0.00000,0.0,73.7
15,6,0,3,2,25.67,39826.0,0.5,18.67,25.0,30.0,5348.0,927510.0,2277.0,17802.0,0.00000,0.00000,0.0,80.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10794,5275,0,0,0,26.00,71716.0,3.0,16.00,24.0,26.0,6843.0,1146280.0,0.0,0.0,1.03993,1.57146,1.0,78.0
10798,5282,0,4,5,15.00,0.0,0.0,10.00,29.0,42.0,7851.0,0.0,0.0,0.0,1.13549,1.32679,1.0,66.9
10800,5287,0,4,5,4.00,0.0,0.5,3.00,29.0,50.0,5646.0,0.0,0.0,0.0,1.58312,1.23487,0.0,78.9
10802,5295,0,4,5,18.00,0.0,0.5,11.00,29.0,28.0,5928.0,0.0,0.0,0.0,1.16317,1.31903,1.0,75.5


## Train-test split

In [21]:
X = data.drop(columns=['VISCODE', 'DX_bl']).to_numpy()
y = data['DX_bl'].to_numpy()

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

## Classifier

In [23]:
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [24]:
y_pred = dtc.predict(X_test)
accuracy = sum(y_pred == y_test) / len(y_test)
print('DTC accuracy:', accuracy)

DTC accuracy: 0.9075144508670521


In [25]:
bag_clf = BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=100, max_samples=0.3, random_state=0)
bag_clf.fit(X_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='gini',
                                                        max_depth=None,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort='deprecated',
                                                        random_state=None,


In [26]:
y_pred = bag_clf.predict(X_test)
accuracy = sum(y_pred == y_test) / len(y_test)
print('DTC accuracy:', accuracy)

DTC accuracy: 0.9393063583815029
